In [1]:
import csv
import numpy as np
import pandas as pd

import data
# Word2vec
from gensim.models import Word2Vec
from gensim.utils import tokenize
from unidecode import unidecode

2022-12-06 20:35:49.934976: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-06 20:35:50.022153: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-06 20:35:50.044930: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-06 20:35:50.457547: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [2]:
train_data = pd.read_csv("data/train.csv")

In [3]:
# embedding dimension
n_dim = 32

In [4]:
sentences = [list(tokenize(s, deacc=True, lower=True)) for s in train_data['text']]
print(sentences[:5])

[['rt', 'refarcir', 'macron', 'ans', 'nom', 'prepare'], ['populaire'], ['faut', 'degager', 'cingle'], ['enseignants', 'mettre', 'prescriptions', 'president', 'republique', 'faut', 'croire', 'president', 'donner', 'periode', 'compte', 'mesures', 'appliquees', 'grever'], ['mafieuse', 'oppressive', 'macron']]


In [5]:
model = Word2Vec(vector_size=n_dim, window=5, min_count=3, workers=7)
model.build_vocab(sentences)
model.train(sentences, total_examples=model.corpus_count, epochs=4)

(11758500, 13100508)

In [6]:
embeddings = np.zeros((train_data.shape[0], n_dim))

In [7]:
found, not_found = [], []
usage = []

In [8]:
for idx in range(train_data.shape[0]):
    curr_usage = 0
    for s in train_data.loc[idx, "text"].split():
        try:
            if len(s) >= 3:
                embeddings[idx] += model.wv[unidecode(s).lower()]
                found.append(s)
                curr_usage += 1
        except:
            not_found.append(s)
            pass
            # print(idx, unidecode(s).lower())
    usage.append((curr_usage, len(train_data.loc[idx, "text"].split())))

In [9]:
print(found[:100], not_found[:100])

['macron', 'ans', 'nom', 'prépare', 'populaire', 'faut', 'dégager', 'cinglé', 'enseignants', 'mettre', 'prescriptions', 'président', 'république', 'faut', 'croire', 'président', 'donner', 'période', 'compte', 'mesures', 'appliquées', 'grever', 'mafieuse', 'oppressive', 'macron', 'équipe', 'campagne', 'macron', 'prie', 'soutien', 'sarko', 'bizarre', 'bizarre', 'cousin', 'sensés', 'dems', 'votent', 'pauvre', 'envoies', 'macron', 'chameau', 'cheval', 'dessiné', 'commission', 'emmanuel', 'macron', 'annule', 'venue', 'soirée', 'électorale', 'bfmtv', 'chaîne', 'prend', 'décision', 'radicale', 'violer', 'pauvre', 'ukrainienne', 'africains', 'düsseldorf', 'petites', 'minutes', 'temps', 'temps', 'bfm', 'heures', 'offre', 'marcheurs', 'roi', 'connard', 'dégoûte', 'poutou', 'continuer', 'battre', 'gonfler', 'intentions', 'vote', 'quinquennat', 'macron', 'série', 'ininterrompue', 'plans', 'com', 'apparence', 'passe', 'melenchon', 'jamais', 'insulter', 'macron', 'publiquement', 'plateau', 'remarque

In [10]:
print(len(found))
print(len(not_found))

3094170
70814


In [11]:
print(usage[:100])

[(4, 6), (1, 1), (3, 3), (14, 14), (3, 3), (13, 13), (6, 6), (11, 11), (5, 6), (14, 14), (3, 3), (7, 9), (2, 2), (17, 19), (6, 7), (10, 10), (4, 4), (2, 2), (8, 8), (8, 8), (4, 4), (4, 4), (4, 5), (16, 16), (18, 19), (13, 13), (18, 18), (6, 6), (10, 10), (19, 19), (4, 4), (2, 2), (9, 12), (12, 14), (8, 8), (11, 11), (13, 13), (13, 13), (8, 8), (6, 7), (7, 7), (18, 18), (4, 4), (7, 7), (14, 14), (14, 15), (7, 7), (12, 12), (8, 10), (3, 3), (5, 5), (19, 19), (11, 11), (6, 7), (1, 1), (16, 18), (3, 3), (9, 13), (4, 4), (12, 12), (12, 12), (6, 6), (6, 7), (8, 8), (1, 1), (1, 2), (2, 2), (3, 4), (12, 13), (13, 14), (4, 4), (5, 5), (4, 7), (10, 10), (5, 5), (8, 9), (5, 5), (17, 17), (8, 9), (11, 11), (8, 8), (3, 3), (4, 4), (3, 4), (18, 18), (5, 6), (9, 9), (6, 8), (4, 4), (13, 13), (6, 7), (6, 7), (7, 7), (14, 15), (4, 4), (16, 16), (13, 14), (4, 4), (2, 3), (6, 6)]


In [12]:
df_new = pd.concat([train_data, pd.DataFrame(embeddings)], axis=1)

In [13]:
display(df_new)

,text,retweets_count,favorites_count,followers_count,statuses_count,friends_count,mentions,urls,verified,hashtags,...,22,23,24,25,26,27,28,29,30,31
0,rt refarcir macron ans nom prépare,3,0,3682,453535,3628,[],[],0,[],...,4.646532,-0.906124,2.260742,3.168564,-1.401000,-1.951112,-2.020051,-2.999555,-1.528810,-2.534579
1,populaire,0,0,86,1016,284,[],[],0,[],...,0.303377,-0.955376,1.113358,0.951807,-2.460639,-0.231257,-0.371027,-0.499201,-0.625621,2.514381
2,faut dégager cinglé,3,1,1944,28234,1995,[],[],0,[],...,4.507827,1.435595,0.588664,0.873489,-4.487713,2.014378,-0.171286,-2.959697,-2.425393,-4.573173
3,enseignants mettre prescriptions président rép...,0,0,1,1072,0,[],['https://t.co/rytlted08g'],0,[],...,11.439668,0.811828,-7.714990,8.646625,-8.791422,1.476102,-5.360216,-6.502421,-18.691670,-17.289281
4,mafieuse oppressive macron,0,0,13957,25311,10841,[],[],0,[],...,1.261369,-0.534618,1.683347,0.508542,-2.935206,-1.011417,0.244173,-1.462812,-0.624479,-0.903896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353964,gonflette tour raciste frustré,0,0,34,1509,55,[],['https://t.co/pma33zhslx'],0,[],...,1.942423,0.090860,0.642374,-1.203738,-8.320486,2.128591,2.228518,-3.868133,-0.779345,0.572256
353965,france caste crapuleuse encadrée gangsters irr...,0,0,89,11166,127,[],[],0,[],...,4.546421,-4.839086,2.855232,4.320019,-3.034552,-1.043354,0.067186,-5.281217,-2.535276,-0.906507
353966,eric zemmour français berbère,3,0,1888,712,3086,[],[],0,[],...,8.608120,-4.907044,5.765294,-1.774291,-9.106257,0.767960,4.852843,-7.070803,1.484933,-2.572688
353967,gauchistes dépression pq,0,0,139,486,320,[],[],0,[],...,0.255871,-1.879965,-0.889584,0.561174,-2.839007,1.631688,1.503662,-0.682619,-0.802378,0.501271


In [14]:
file_path = f"data/train_with_embeddings.csv"

In [15]:
df_new.to_csv(file_path)

In [16]:
eval_data = pd.read_csv("data/evaluation.csv")

In [17]:
embeddings = np.zeros((eval_data.shape[0], n_dim))

In [18]:
for idx in range(eval_data.shape[0]):
    for s in eval_data.loc[idx, "text"].split():
        try:
            embeddings[idx] += model.wv[unidecode(s).lower()]
        except:
            pass
            # print(idx, unidecode(s).lower())

In [19]:
df_new = pd.concat([eval_data, pd.DataFrame(embeddings)], axis=1)

In [20]:
display(df_new)

,text,favorites_count,followers_count,statuses_count,friends_count,mentions,urls,verified,hashtags,timestamp,...,22,23,24,25,26,27,28,29,30,31
0,descendue pourcentage vote,0,85,4442,327,[],[],0,"['presidentielle2022', 'hidalgo']",1647607994000,...,0.236177,1.684259,0.337494,-1.841154,-2.819763,-3.362540,0.566980,0.270283,-1.904030,0.906213
1,photo demande arbre généalogique,0,427,33282,443,[],[],0,[],1647601275000,...,1.224501,-1.906678,-0.938039,0.172506,-1.731693,2.296625,-1.628562,1.963228,1.429320,-3.019880
2,dossiers promis poste gars vraiment fiable,6,1127,13111,1596,[],[],0,['macron'],1646900684000,...,6.140955,1.323814,0.087359,1.188006,-2.537488,4.407669,-2.399221,-2.754585,-3.754223,-2.722256
3,lol cas faut second tour macron sortir définit...,2,1699,25760,2036,[],['https://t.co/7ox54d8dlm'],0,[],1647161294000,...,17.498280,9.785352,8.969147,-2.368792,-25.098829,-11.513661,1.673871,-13.322234,-4.362976,-6.330258
4,zemmour putain discours propagandiste littéral...,0,249,20718,369,[],"['https://t.co/tsw9cg6v9j', 'https://t.co/t0j9...",0,[],1647724874000,...,13.178494,-8.557063,-0.358623,10.207128,-22.118301,6.637660,-3.199759,-14.670279,-3.621372,-7.693866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117985,balais inexcusable vote veut,0,4,194,64,[],[],0,[],1647103416000,...,1.732712,-0.234866,1.961473,0.457789,-3.606581,-2.538189,-1.665566,-1.775454,-2.409455,-2.009746
117986,balayant rap borloo mâles blcs commis faute fo...,0,744,39489,894,[],['https://t.co/im5cs5ur1a'],0,[],1647490899000,...,15.875614,-4.698699,1.880658,9.216993,-7.924570,-1.044077,-8.604853,-4.796047,-12.932811,-5.407116
117987,triste réalité covid tue malades macron veille...,12,924,2270,159,[],[],0,[],1647522384000,...,8.539811,-4.205772,2.478307,9.407774,-6.229049,-4.018029,-2.941662,-12.503990,-7.855281,-8.272663
117988,excellente idée pouvez réussir démarche crise ...,0,5,4,80,[],['https://t.co/xontgm06pb'],0,[],1647607262000,...,16.175381,-8.453264,0.512585,11.065447,-2.040199,-0.690955,-3.653930,-4.793446,-14.500380,-11.265319


In [21]:
file_path = f"data/evaluation_with_embeddings.csv"

In [22]:
df_new.to_csv(file_path)